In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv("Alexandre Marie de ficquelmont- Albert School - B2 S1 - Data set_ LTV modelling for Birchbox - Albert School - B2 S1 - Data set_ LTV modelling for Birchbox - Feuille 1.csv")

In [4]:
df

,Customer ID,Order date,Order value,Product contained in the order
0,555c03bf-5c9e-44dd-bd2e-80fa0268416a,2023-12-27 9:31:32,"137,02",Mascara
1,dffd1355-a413-421a-ad75-ca0f16707f17,2023-05-25 15:30:25,"182,48",Blush
2,1f672088-17ef-4816-be6e-6845bc97d93a,2024-07-30 19:49:38,"185,13",Concealer
3,51b0a3c5-c9e7-4aba-b5da-3e50822bbc3d,2021-12-13 21:19:24,"146,71",Concealer
4,414ac07f-5968-43e9-9d76-699d426de098,2024-01-24 19:31:20,"130,68",Lipstick
...,...,...,...,...
34541,9f2d5db8-deba-489f-959d-e20409642b31,2023-03-15 9:25:59,"112,82",Eyeliner
34542,5a4f4145-fc98-4279-8f6f-111c26c06e67,2023-05-27 17:46:17,"73,65",Eyeshadow
34543,2cbcf132-d9b4-4fa5-89e7-fddc9ebb8f60,2023-05-01 7:20:11,"191,62",Eyeliner
34544,33dd5b01-cf3e-434f-8c05-206a4b620b4d,2024-03-17 12:15:22,"123,42",Foundation


In [5]:
import numpy as np

col_map = {
    'Customer ID': 'customer_id',
    'Order date': 'order_date',
    'Order value': 'order_value',
    'Product contained in the order': 'product'
}
df = df.rename(columns=col_map)

df['order_date'] = pd.to_datetime(df['order_date'])
val = df['order_value'].astype(str).str.replace(',', '.', regex=False)
df['order_value'] = pd.to_numeric(val, errors='coerce')

df = df[df['order_date'] >= '2021-01-01'].copy()

df['order_month'] = df['order_date'].values.astype('datetime64[M]')

first_order = df.groupby('customer_id', as_index=False)['order_month'].min().rename(columns={'order_month': 'acq_month'})
df = df.merge(first_order, on='customer_id', how='left')

cohort_sizes = first_order.groupby('acq_month', as_index=False).size().rename(columns={'size': 'cohort_size'})

cohort_sizes.head()


,acq_month,cohort_size
0,2021-01-01,50
1,2021-02-01,1101
2,2021-03-01,716
3,2021-04-01,490
4,2021-05-01,478


In [6]:
first_order_date = df.groupby('customer_id', as_index=False)['order_date'].min() \
    .rename(columns={'order_date': 'first_order_date'})

df = df.merge(first_order_date, on='customer_id', how='left')
df['is_first_order'] = df['order_date'] == df['first_order_date']

df.head()


,customer_id,order_date,order_value,product,order_month,acq_month,first_order_date,is_first_order
0,555c03bf-5c9e-44dd-bd2e-80fa0268416a,2023-12-27 09:31:32,137.02,Mascara,2023-12-01,2021-07-01,2021-07-15 18:31:12,False
1,dffd1355-a413-421a-ad75-ca0f16707f17,2023-05-25 15:30:25,182.48,Blush,2023-05-01,2021-04-01,2021-04-25 06:13:16,False
2,1f672088-17ef-4816-be6e-6845bc97d93a,2024-07-30 19:49:38,185.13,Concealer,2024-07-01,2021-02-01,2021-02-13 20:17:21,False
3,51b0a3c5-c9e7-4aba-b5da-3e50822bbc3d,2021-12-13 21:19:24,146.71,Concealer,2021-12-01,2021-12-01,2021-12-13 21:19:24,True
4,414ac07f-5968-43e9-9d76-699d426de098,2024-01-24 19:31:20,130.68,Lipstick,2024-01-01,2021-08-01,2021-08-03 20:46:19,False


In [7]:

def month_diff(a, b):
    return (a.year - b.year) * 12 + (a.month - b.month)

rel = df[['customer_id','acq_month','order_month','order_value']].copy()
rel['period'] = rel.apply(lambda r: month_diff(r['order_month'], r['acq_month']), axis=1)

revenue_by_cohort = rel.groupby(['acq_month','period'], as_index=False)['order_value'].sum().rename(columns={'order_value':'revenue'})

revenue_by_cohort = revenue_by_cohort.merge(cohort_sizes, on='acq_month', how='left')

revenue_by_cohort.head(10)


,acq_month,period,revenue,cohort_size
0,2021-01-01,0,5105.67,50
1,2021-01-01,1,1003.04,50
2,2021-01-01,2,554.10,50
3,2021-01-01,3,547.07,50
4,2021-01-01,4,204.17,50
5,2021-01-01,5,458.13,50
6,2021-01-01,6,624.35,50
7,2021-01-01,7,373.71,50
8,2021-01-01,8,281.54,50
9,2021-01-01,9,107.65,50


In [8]:

arpu = revenue_by_cohort.copy()
arpu['arpu'] = arpu['revenue'] / arpu['cohort_size']

arpu = arpu.sort_values(['acq_month','period'])
arpu['arpu_cum'] = arpu.groupby('acq_month')['arpu'].cumsum()

arpu_cum_pivot = arpu.pivot(index='acq_month', columns='period', values='arpu_cum').sort_index()

arpu_cum_pivot.head(12)


period,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
acq_month,,,,,,,,,,,,,,,,,,,,,
2021-01-01,102.113400,122.174200,133.256200,144.197600,148.281000,157.443600,169.930600,177.404800,183.035600,185.188600,...,439.459800,454.718800,473.807800,482.560600,491.281200,497.053600,NaN,504.718800,521.710200,524.747
2021-02-01,116.748374,125.477003,132.632016,139.619055,147.636903,157.461063,164.545704,170.480509,176.452879,183.714051,...,410.371535,424.408038,434.059991,444.346930,451.166076,458.767402,466.783460,473.287965,480.484005,NaN
2021-03-01,114.626257,120.321327,126.577598,133.652249,141.530307,150.302263,155.923813,161.108589,168.452137,180.536047,...,411.054008,419.462151,432.558380,438.579679,444.417500,451.862500,461.715042,469.493925,NaN,NaN
2021-04-01,113.148265,121.385510,128.105286,138.266490,146.844551,152.258571,158.117673,165.545959,178.631408,193.529714,...,407.531633,416.602327,422.712714,429.813857,440.013184,446.076020,454.639735,NaN,NaN,NaN
2021-05-01,116.106506,125.404979,134.758661,143.396067,149.843598,154.815858,162.101360,173.354121,187.968326,202.520628,...,415.218013,419.368368,428.006778,435.483347,444.212803,451.491611,NaN,NaN,NaN,NaN
2021-06-01,112.744262,120.238167,128.174310,133.863071,141.132810,147.734500,159.973905,173.461452,184.681738,193.068119,...,406.514238,414.769310,423.972119,429.108262,439.357929,NaN,NaN,NaN,NaN,NaN
2021-07-01,118.311966,125.623179,130.560444,135.460308,144.122889,156.906769,169.669795,183.942855,192.441744,199.857726,...,420.782393,427.999949,433.419641,439.470085,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-01,117.637494,124.763212,132.070462,139.842822,154.196861,168.894161,182.799343,190.039878,196.707397,205.460073,...,419.281484,429.110073,439.911144,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-01,110.537331,121.342830,125.274084,138.887106,147.784984,157.259839,167.226592,176.548585,186.045949,192.235723,...,397.604277,405.648650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:

weights = arpu[['acq_month','period','cohort_size','arpu_cum']].copy()
weighted = (weights['arpu_cum'] * weights['cohort_size'])
weighted_arpu_cum = weights.groupby('period').apply(
    lambda g: (g['arpu_cum'] * g['cohort_size']).sum() / g['cohort_size'].sum()
).rename('weighted_arpu_cum').reset_index()

weighted_arpu_cum.head(12)


C:\Users\moham\AppData\Local\Temp\ipykernel_36156\1423533780.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_arpu_cum = weights.groupby('period').apply(


,period,weighted_arpu_cum
0,0,114.749038
1,1,122.940590
2,2,130.249232
3,3,137.683537
4,4,145.670229
5,5,153.574438
6,6,161.548666
7,7,169.356894
8,8,177.010792
9,9,185.341965


In [ ]:
import matplotlib.pyplot as plt
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

wa = weighted_arpu_cum.dropna().copy()
X = wa['period'].values.reshape(-1,1)
y = wa['weighted_arpu_cum'].values

iso = IsotonicRegression(increasing=True, out_of_bounds='clip')
iso.fit(wa['period'].values, y)

poly2 = Pipeline([
    ('poly', PolynomialFeatures(2, include_bias=False)),
    ('lin', LinearRegression())
])
poly2.fit(X, y)

horizon = np.arange(0, 61)
pred_iso = iso.predict(horizon)
pred_poly2 = poly2.predict(horizon.reshape(-1,1))
pred_cum = np.minimum(pred_iso, pred_poly2)

ltv_48 = float(pred_cum[48])
ltv_60 = float(pred_cum[60])

plt.figure(figsize=(12,6))
plt.plot(wa['period'], wa['weighted_arpu_cum'], 'o', label="ARPU cumulé pondéré (données)")
plt.plot(horizon, pred_iso, '--', label="IsotonicRegression")
plt.plot(horizon, pred_poly2, ':', label="Régression polynomiale degré 2")
plt.plot(horizon, pred_cum, '-', label="Min des deux modèles")
plt.axvline(48, color='grey', linestyle='--', alpha=0.5)
plt.axhline(ltv_48, color='red', linestyle='--', alpha=0.3, label=f"LTV 48m = {ltv_48:.2f}")
plt.axvline(60, color='grey', linestyle='--', alpha=0.5)
plt.axhline(ltv_60, color='purple', linestyle='--', alpha=0.3, label=f"LTV 60m = {ltv_60:.2f}")

plt.xlabel("Période (mois depuis l'acquisition)")
plt.ylabel("ARPU cumulé pondéré")
plt.title("Projection du LTV client (ARPU cumulé pondéré dans le temps)")
plt.legend()
plt.grid(True)
plt.show()

ltv_48, ltv_60


(524.747, 524.747)

In [11]:
import ipywidgets as widgets
from IPython.display import display


def precompute_arpu_cum(df):
    grouped = {}
    grouped["(Tous)"] = df.copy()
    for p in sorted(df['product'].dropna().unique()):
        grouped[p] = df[df['product']==p].copy()
    results = {}
    for k,df_f in grouped.items():
        first_order_f = df_f.groupby('customer_id', as_index=False)['order_month'].min().rename(columns={'order_month': 'acq_month'})
        cohort_sizes_f = first_order_f.groupby('acq_month', as_index=False).size().rename(columns={'size':'cohort_size'})
        rel_f = df_f[['customer_id','acq_month','order_month','order_value']].copy()
        rel_f = rel_f.merge(first_order_f, on='customer_id', how='left')
        
        rel_f['period'] = rel_f.apply(lambda r: month_diff(r['order_month'], r['acq_month_y']), axis=1)
        
        revenue_f = rel_f.groupby(['acq_month_y','period'], as_index=False)['order_value'].sum().rename(columns={'acq_month_y':'acq_month','order_value':'revenue'})
        revenue_f = revenue_f.merge(cohort_sizes_f, on='acq_month', how='left')
        arpu_f = revenue_f.copy()
        arpu_f['arpu'] = arpu_f['revenue'] / arpu_f['cohort_size']
        arpu_f = arpu_f.sort_values(['acq_month','period'])
        arpu_f['arpu_cum'] = arpu_f.groupby('acq_month')['arpu'].cumsum()
        results[k] = arpu_f
    return results

arpu_cum_cache = precompute_arpu_cum(df)

products = list(arpu_cum_cache.keys())
product_dd = widgets.Dropdown(options=products, value='(Tous)', description='Produit:')

@widgets.interact(produit=product_dd)
def show_arpu_cum(produit):
    arpu_f = arpu_cum_cache[produit]
    pivot_f = arpu_f.pivot(index='acq_month', columns='period', values='arpu_cum').sort_index()
    display(pivot_f.head(20))


interactive(children=(Dropdown(description='Produit:', options=('(Tous)', 'Blush', 'Concealer', 'Eyeliner', 'E…

In [12]:


def build_arpu_cum_for(df_sub):
    first_order_p = df_sub.groupby('customer_id', as_index=False)['order_month'].min().rename(columns={'order_month': 'acq_month'})
    cohort_sizes_p = first_order_p.groupby('acq_month', as_index=False).size().rename(columns={'size':'cohort_size'})

    rel_p = df_sub[['customer_id','acq_month','order_month','order_value']].copy()
    rel_p = rel_p.merge(first_order_p, on='customer_id', how='left')
    rel_p['period'] = rel_p.apply(lambda r: month_diff(r['order_month'], r['acq_month_y']), axis=1)

    revenue_p = rel_p.groupby(['acq_month_y','period'], as_index=False)['order_value'].sum().rename(columns={'acq_month_y':'acq_month','order_value':'revenue'})
    revenue_p = revenue_p.merge(cohort_sizes_p, on='acq_month', how='left')

    arpu_p = revenue_p.copy()
    arpu_p['arpu'] = arpu_p['revenue'] / arpu_p['cohort_size']
    arpu_p = arpu_p.sort_values(['acq_month','period'])
    arpu_p['arpu_cum'] = arpu_p.groupby('acq_month')['arpu'].cumsum()
    return arpu_p

rows = []
for prod in sorted(df['product'].dropna().unique().tolist()):
    dfp = df[df['product']==prod]
    arpu_p = build_arpu_cum_for(dfp)
    def wavg(arpu_tbl, p):
        g = arpu_tbl[arpu_tbl['period']==p]
        if g.empty:
            return np.nan
        return (g['arpu_cum']*g['cohort_size']).sum()/g['cohort_size'].sum()
    ltv_1m = wavg(arpu_p, 1)
    ltv_24m = wavg(arpu_p, 24)
    ratio = np.nan
    if pd.notnull(ltv_1m) and pd.notnull(ltv_24m) and ltv_1m!=0:
        ratio = ltv_24m/ltv_1m
    rows.append({'product': prod, 'ltv_1m': ltv_1m, 'ltv_24m': ltv_24m, 'ratio_24_1': ratio})

recap = pd.DataFrame(rows).sort_values('ratio_24_1', ascending=False)

best_prod = recap.iloc[0]['product'] if not recap.empty else None

recap, best_prod


(      product      ltv_1m     ltv_24m  ratio_24_1
 5    Lipstick  113.515702  138.173487    1.217219
 7      Powder  113.202826  137.131360    1.211378
 1   Concealer  111.004160  133.083701    1.198907
 4  Foundation  112.658437  134.419522    1.193160
 6     Mascara  113.693587  135.404418    1.190959
 0       Blush  111.684694  131.025589    1.173174
 2    Eyeliner  113.178867  132.269504    1.168677
 3   Eyeshadow  111.317196  129.533553    1.163644,
 'Lipstick')

### Analyse et interprétation

- **ARPU cumulée et réachat**: si la courbe ARPU cumulée pondérée croît rapidement au début puis ralentit, cela suggère un **réachat fort en early lifecycle** puis une **stabilisation/attrition**. Une pente quasi-linéaire dans le temps indiquerait des réachats réguliers, tandis qu’un palier précoce traduirait un **churn rapide** après l’acquisition.
- **Modèle LTV (4 et 5 ans)**: l’enveloppe conservatrice entre régression isotone (monotone) et polynomiale limite la **sur‑extrapolation**. Si la trajectoire projetée s’aplatit avant 48–60 mois, le **potentiel incrémental long terme** est modéré; si la pente reste positive, il existe une **tail réserve** de revenus.

#### Hypothèses (3–4) expliquant les tendances d’ARPU observées
1. **Promotions d’acquisition**: un pic initial d’ARPU (mois 0–1) suivi d’un plateau peut venir d’offres d’entrée attirant des clients **prix‑sensibles** qui ne répètent pas.
2. **Mix produit d’essai**: si l’ARPU se maintient grâce aux **consommables** (p. ex. mascara, eyeliner), cela reflète des **cycles de réachat** naturels; une part plus forte de produits durables réduirait la pente.
3. **Calendrier marketing**: campagnes saisonnières (Noël, soldes) créent des **sur‑pentes** périodiques; hors saison, la croissance ralentit.
4. **Expérience et rétention**: si la pente baisse sur les cohortes récentes, cela peut indiquer **friction UX/logistique** (livraison, SAV) ou **concurrence accrue**, augmentant l’attrition.

### Aller plus loin: ratio LTV(24m)/LTV(1m)
- **Interprétation**: ce ratio mesure l’**expansion au‑delà du premier mois**. Un ratio élevé indique une **forte rétention et/ou montée en panier** après M1; un ratio faible signale **essoufflement** rapide.
- **Produit au meilleur ratio**: mis en évidence dans la table `recap` (variable `best_prod`). Cela suggère un **comportement de réachat** supérieur et/ou une **cross‑sell** associée à ce produit.

#### 3 hypothèses expliquant le plus haut ratio et plan d’analyses
1. **Produit consommable à fréquence élevée**: nécessite remplacements réguliers.
   - Analyses: distribution des **intervalles de réachat**, taux de **répétition à 30/60/90j**, courbe d’**hazard**.
2. **Effet halo/cross‑sell**: ce produit entraîne des achats d’autres catégories.
   - Analyses: **baskets** post‑achat, **séquences** d’achats, uplift **attach rate** vs autres produits.
3. **Cohortes d’acquisition de meilleure qualité**: canal/prix attirant des clients plus fidèles pour ce produit.
   - Analyses: LTV par **mois/cohorte d’acquisition**, **segmentation** par ticket initial, **survie** par cohorte.

Notes pratiques:
- Les tables produites incluent: revenu par cohorte/période, **taille de cohorte**, **ARPU cumulée** par cohorte, **ARPU cumulée pondérée** (série globale). Le **dropdown produit** permet de filtrer.
- La projection LTV (48/60 mois) se lit dans la cellule de modélisation (`ltv_48`, `ltv_60`).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

wa_clean = (
    weighted_arpu_cum
        .assign(
            period=pd.to_numeric(weighted_arpu_cum['period'], errors='coerce'),
            weighted_arpu_cum=pd.to_numeric(weighted_arpu_cum['weighted_arpu_cum'], errors='coerce')
        )
        .dropna(subset=['period', 'weighted_arpu_cum'])
        .copy()
)

wa_clean['period'] = wa_clean['period'].astype(int)
wa_clean = wa_clean.sort_values('period').drop_duplicates(subset=['period'], keep='last')

if wa_clean.empty or wa_clean['period'].nunique() == 0:
    raise ValueError("Aucune donnée propre pour la régression (toutes NaN ou non numériques)")

X = wa_clean['period'].values.reshape(-1, 1)
y = wa_clean['weighted_arpu_cum'].values

# 2) Régression isotone (monotone)
iso = IsotonicRegression(increasing=True, out_of_bounds='clip')
iso.fit(wa_clean['period'].values, y)

# 3) Régression polynomiale avec fallback de degré
unique_periods = wa_clean['period'].nunique()
degree = 2 if unique_periods >= 3 else 1
poly = Pipeline([
    ('poly', PolynomialFeatures(degree, include_bias=False)),
    ('lin', LinearRegression())
])
poly.fit(X, y)

horizon = np.arange(0, 61)
pred_iso = iso.predict(horizon)
pred_poly = poly.predict(horizon.reshape(-1, 1))
pred_cum = np.minimum(pred_iso, pred_poly)

ltv_48 = float(pred_cum[48]) if len(pred_cum) > 48 else np.nan
ltv_60 = float(pred_cum[60]) if len(pred_cum) > 60 else np.nan

plt.figure(figsize=(12, 6))
plt.plot(wa_clean['period'], wa_clean['weighted_arpu_cum'], 'o', label="ARPU cumulé pondéré (données)")
plt.plot(horizon, pred_iso, '--', label="IsotonicRegression")
plt.plot(horizon, pred_poly, ':', label=f"Régression polynomiale degré {degree}")
plt.plot(horizon, pred_cum, '-', label="Min des deux modèles")

if not np.isnan(ltv_48):
    plt.axvline(48, color='grey', linestyle='--', alpha=0.5)
    plt.axhline(ltv_48, color='red', linestyle='--', alpha=0.3, label=f"LTV 48m = {ltv_48:.2f}")
if not np.isnan(ltv_60):
    plt.axvline(60, color='grey', linestyle='--', alpha=0.5)
    plt.axhline(ltv_60, color='purple', linestyle='--', alpha=0.3, label=f"LTV 60m = {ltv_60:.2f}")

plt.xlabel("Période (mois depuis l'acquisition)")
plt.ylabel("ARPU cumulé pondéré")
plt.title("Projection LTV robuste (nettoyage NaN, fallback polynomial)")
plt.legend()
plt.grid(True)
plt.show()

ltv_48, ltv_60

arpu_safe = arpu.copy()
arpu_safe = arpu_safe[arpu_safe['period'].notna()].copy()
arpu_safe['period'] = pd.to_numeric(arpu_safe['period'], errors='coerce')
arpu_safe = arpu_safe.dropna(subset=['period']).copy()
arpu_safe['period'] = arpu_safe['period'].astype(int)
arpu_safe = arpu_safe.sort_values(['acq_month', 'period'])

pivot_safe = arpu_safe.pivot(index='acq_month', columns='period', values='arpu_cum').sort_index()
pivot_safe = pivot_safe.replace([np.inf, -np.inf], np.nan)

# Affiche un aperçu des 20 premières lignes pour contrôle
pivot_safe.head(20)
